# Installations and Definitions

Run the below cells for installations and imports.

In [ ]:
pip install cvxpy --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 9.1 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [ ]:
# Run this cell for all the imports

import numpy as np
import cvxpy as cp


import scipy as sp
import random
import time
from matplotlib import pyplot as plt

np.set_printoptions(precision=4) # Comment this line to see more than 4 digits after decimal point

from numpy import linalg as la
from scipy import linalg as sla
from scipy.linalg import sqrtm as sqm
from numpy import trace as tr
from numpy import allclose as ac
from numpy.linalg import eigh as eigh
from numpy.linalg import norm as norm


plt.style.use('ggplot')

Run the following cells to define the required functions.

In [ ]:
# # Returns a complex Positive semidefinite matrix.
# def PSDMatrix(Dim, IsDensity = 1, Rank = 1):

#   #Define a random square matrix of size Dim
#   X = 2*np.matrix(np.random.randn(Dim, Rank)) - 1 + 1j*(2*np.matrix(np.random.randn(Dim, Rank)) - 1)
#   #X = 2*np.matrix(np.random.rand(Dim, DimCol)) - 1
  
#   #Multiply by its transpose to obtain a postive semidefinite matrix
#   P = X@(X.conj().T)
  
#   return P 

In [ ]:
# Dim: dimension
# Rank: Rank. For full rank matrix, set Rank = Dim
# IsReal: Set IsReal = 0 for complex valued matrices and 1 for real-valued matrices

def DensityMatrix(Dim, Rank, IsReal = 0):

  #Define a random square matrix of size Dim
  if IsReal == 1:
    X = np.matrix(np.random.randn(Dim, Rank)) + 1.5 #1.5 is a bias added to ensure the mean of the distribution is not the maximally mixed state
  else:
    X = np.matrix(np.random.randn(Dim, Rank)) + 1j*(np.matrix(np.random.randn(Dim, Rank))) + 1.5
  
  
  #Multiply by its transpose to obtain a postive semidefinite matrix
  P = X@(X.conj().T)

  #Normalise to obtain density matrix
  P = P/np.trace(P)

  return P 

In [ ]:
# Return an random n dimensional probability vector
def ProbVec(n):

  p = np.random.rand(n)
  return p/np.sum(p)

In [ ]:
# Generates n d-dimensional r-rank states
# Choose IsReal = 1 for Real valued matrices
def GenStates(n, d, r, IsReal = 0):

  Rhos = np.empty((n,d,d)) + 1j*np.empty((n,d,d)) 
  for i in range(n):
    Rhos[i,:,:] = DensityMatrix(d,r, IsReal)

  return Rhos

In [ ]:
# Given a collection of states and a probability vector, the function returns
# RhoAv: Average state

def AverageState(Rhos, p):
  
  (n,d) = np.shape(Rhos)[0:2]

  RhoAv = np.zeros((d,d)) + 1j*np.zeros((d,d)) 
  
  for i in range(n):
    RhoAv += p[i]*Rhos[i,:,:]



  return (RhoAv)

In [ ]:
# Returns the square root Fidelity between two PSD operators P and Q
def Fidelity(P, Q):
  
  sqP = sqm(P)

  return(tr(sqm(sqP@Q@sqP)))

In [ ]:
# # Returns the truth value of a matrix A being Hermitian
# def IsHerm(A):
#   return(ac(A, A.conj().T))

In [ ]:
# # Returns the truth value of a matrix U being Unitary
# def IsUnitary(U):

#   d = np.shape(U)[0]

#   return(ac(U@U.conj().T, np.eye(d)) and ac(U.conj().T@U, np.eye(d)))

In [ ]:
# Given an ensemble of states Rhos and a probability vector p over it, returns
# the average fidelity \Sum_i p(i)F(Rhos(i), Sigma) of Sigma

def AverageFidelity(Sigma, Rhos, p):

  (n,d) = np.shape(Rhos)[0:2]

  Fid = 0
  for i in range(n):
    Fid = Fid + p[i]*Fidelity(Rhos[i], Sigma)

  return (Fid)

In [ ]:
# Returns the commuting estimator of the ensemble (Rhos, p) 
def CommEstimator(Rhos, p):

  (n,d) = np.shape(Rhos)[0:2]
  Sigma = 0*Rhos[0]

  for i in range(n):
    Sigma += p[i]*sqm(Rhos[i])
  
  Sigma = Sigma@Sigma
  Sigma = Sigma/tr(Sigma)

  return(Sigma)

In [ ]:
# Returns the product bound of the ensemble (Rhos, p)
def ProductBound(Rhos, p):

  n = np.shape(Rhos)[0]


  ProdBound = 0
  for i in range(n):
    ProdBound += p[i]**2
    for j in range(n):
      if i < j:
        ProdBound += 2*p[i]*p[j]*Fidelity(Rhos[i], Rhos[j])
 
  ProdBound = np.sqrt(ProdBound)

  return(ProdBound)

In [ ]:
def OmegaIteration(Rhos, p, SigmaIn, n, d, SigmaZero):

  SqrtSigma = sqm(SigmaIn)
  SigmaOut = SigmaZero
  for i in range(n):
    SigmaOut = SigmaOut + p[i]*sqm(SqrtSigma@Rhos[i]@SqrtSigma)

  SigmaInvHalf = la.inv(SqrtSigma) 
  SigmaOut = SigmaInvHalf@SigmaOut@SigmaOut@SigmaInvHalf  
  SigmaOut = SigmaOut/tr(SigmaOut)
  return(SigmaOut)

In [ ]:
# Fixed point algorithm for optimal fidelity estimator

# Returns
# Fid: Optimal average fidelity
# SigmaOut: Optimal estimator
# count: Number of steps for convergence
# Sigmas: States at each step of the iteration process

def OmegaEstimator(Rhos, p, eps = 1e-6):


  Sigmas = []
  (n,d) = np.shape(Rhos)[0:2]
  SigmaOut = np.zeros([d,d]) + 1j*np.zeros([d,d])
  

  SigmaIn = CommEstimator(Rhos, p)
 
    
  Sigmas.append(SigmaIn)
  SigmaZero = np.zeros([d,d]) + 1j*np.zeros([d,d])

  count = 0
  delta = 1
  while delta > eps:
    count += 1
    SigmaOut = OmegaIteration(Rhos,p, SigmaIn, n, d, SigmaZero)
    # Fidelities.append(Fid2)
    delta = la.norm(SigmaIn - SigmaOut, np.inf)
    # delta = 1 - Fidelity(SigmaIn, SigmaOut)
    Sigmas.append(SigmaOut)
    SigmaIn = SigmaOut


  Fid = AverageFidelity(SigmaOut, Rhos, p)
  return(Fid, SigmaOut, count, Sigmas)


In [ ]:
# Carries our a single iteration of Lambda Fixed point algorithm
def LambdaIteration(Rhos, p, SigmaIn, n, d, SigmaZero):

  SqrtSigma = sqm(SigmaIn)
  SigmaOut = SigmaZero
  for i in range(n):
    SigmaOut = SigmaOut + p[i]*sqm(SqrtSigma@Rhos[i]@SqrtSigma)
    
  SigmaOut = SigmaOut/tr(SigmaOut)
  return(SigmaOut)

In [ ]:
# Fixed point algorithm for optimal fidelity estimator
def LambdaEstimator(Rhos, p, eps = 1e-6):

  (n,d) = np.shape(Rhos)[0:2]
  SigmaIn = np.zeros([d,d]) + 1j*np.zeros([d,d])
  SigmaOut = np.zeros([d,d]) + 1j*np.zeros([d,d])
  Fidelities = []

  SigmaIn = CommEstimator(Rhos, p)
      

  SigmaZero = np.zeros([d,d]) + 1j*np.zeros([d,d])
   
  count = 0
  delta = 1
  while delta > eps:
    count += 1
    (SigmaOut) = LambdaIteration(Rhos,p, SigmaIn, n, d, SigmaZero)
    delta = la.norm(SigmaIn - SigmaOut, np.inf)
    # delta = 1 - Fidelity(SigmaIn, SigmaOut)
    SigmaIn = SigmaOut


  Fid = AverageFidelity(SigmaOut, Rhos, p)
  return(Fid, SigmaOut, count, Fidelities)


In [ ]:
# Original SDP for optimal average fidelity

# Uses cvxpy to solve the optimal estimator SDP problem.
# Rhos is an (n,d,d) tensor containing the n d-dimensional states.
# p is an n dimensional probability vector.

# Outputs
# Fid: The primal (and dual) optimum of the problem. This is expected to be the optimal fidelity.
# Sigma: The optimal estimator that produces Fid
# XList: The collection of X operators that constitute the primal optimal operator.
# YList: The collection of Y operators that constitute the dual optimal operator.
# z: The value z which is a part of the dual problem. 

def BayesSDPFidOri(Rhos, p, epsi = 1e-10):
  
  (n,d) = np.shape(Rhos)[0:2]
  Id = np.eye(d)
  Zeros = np.zeros([d,d])

  # A is the primal constraint operator as defined in Watrous
  A = 0.5*np.block([[np.zeros([n*d, n*d]), np.kron(p,Id).T],[np.kron(p.T,Id), Zeros]])

  # Primal feasible
  X = cp.Variable(((n+1)*d, (n+1)*d), complex = True)

  # Constraints
  constraints = [X >> 0]  #PSD
  constraints += [X.H == X] # Hermitian
  for i in range(n):
    constraints += [X[i*d:(i+1)*d,i*d:(i+1)*d] == Rhos[i,:,:]]
  constraints += [cp.trace(X[n*d:n*d+d, n*d:n*d+d]) == 1]

  # Solve problem
  prob = cp.Problem(cp.Maximize(cp.real(cp.trace(A@X))), constraints)
  prob.solve(eps = epsi)
  
  # Extract quantities of interest from primal and dual variables
  Fid = prob.value
  Sigma = X.value[n*d:n*d + d, n*d:n*d + d]
  XList = X.value[0:n*d, n*d: n*d + d]
  XList = XList.reshape(n,d,d)
  YList = np.zeros((n,d,d)) + 1j*np.zeros((n,d,d)) 
  z = constraints[-1].dual_value
  KList = np.zeros((n,d,d)) + 1j*np.zeros((n,d,d))

  for j in range(2, n+2):
    YList[j-2,:,:] = constraints[j].dual_value
  
  for i in range(n):
    if np.linalg.det(Rhos[i]) == 0:
      KList[i] = sp.linalg.sqrtm(np.linalg.pinv(Rhos[i]))@XList[i]@sp.linalg.sqrtm(np.linalg.pinv(Sigma))
    else:   
      KList[i] = sp.linalg.sqrtm(np.linalg.inv(Rhos[i]))@XList[i]@sp.linalg.sqrtm(np.linalg.inv(Sigma))

  XX = X.value

  return (Fid, Sigma, XList, YList, z, KList, XX)

In [ ]:
# Alternate SDP for optimal average fidelity
# Faster than BayesSDPFidOri 
# Outputs:
# Fid: Optmal fidelity
# Sigma: Optimal state
# XList: List of X_i such that Tr(X_i) = F(\Rhos[i], \Sigma)
# UList: List of Unitaries U_i
# z: Dual variable such that Fid = 2z


def BayesSDPFid(Rhos, p, epsi = 1e-6):

  (n,d) = np.shape(Rhos)[0:2]
  Id = np.eye(d)
  Zeros = np.zeros([d,d])

  A = np.zeros([2*d,2*d])
  A[0:d,d:2*d] = 0.5*np.eye(d)
  A[d:2*d, 0:d] = 0.5*np.eye(d)
  
  As = []
  for i in range(n):  
    As.append((p[i])*A)
  
  XXs = []
  for i in range(n):
    XXs.append(cp.Variable((2*d, 2*d), complex = True))
  
  Sig = cp.Variable((d, d), complex = True)
  
  constraints = [cp.trace(Sig) == 1]

  for i in range(n):
    constraints += [XXs[i].H == XXs[i]] # Hermitian
    constraints += [XXs[i] >> 0]  #PSD
    constraints += [XXs[i][0:d, 0:d] == Rhos[i]]
    constraints += [XXs[i][d:2*d, d:2*d] == Sig]
    
  Arg = sum([As[i] @ XXs[i] for i in range(n)])

  prob = cp.Problem(cp.Maximize(cp.real(cp.trace(Arg))), constraints)
  prob.solve(eps = epsi)

  Fid = prob.value
  Sigma = Sig.value
  
  XList = []
  UList = []
  for i in range(n):
    Xi = XXs[i].value[0:d, d: 2*d]
    XList.append(Xi)
    Ui = la.pinv(sqm(Rhos[i]))@Xi@la.inv(sqm(Sigma))
    UList.append(Ui)

  z = constraints[0].dual_value

  return(Fid, Sigma, XList, UList, z)

In [ ]:
# Function to plot Scatter or line plots
# Ease of life function

def PlotScatters(X, Ys, XLabel, YLabel, Title, Colors, Markers, Labels, FigSize, FigName, LineOrScatter, Loc, SaveFig = 0, IsLogScale = 1, IsXTicks = 0, XTicks = []):

  fig, ax = plt.subplots(nrows=1, ncols=1, figsize = FigSize)

  plt.ylabel(YLabel, fontsize = 15)
  plt.xlabel(XLabel, fontsize = 15)
  fig.suptitle(Title, fontsize = 18)

  Iters = len(Ys[0])

  SortedYs = []
  UpYs = []
  DownYs = []

  if LineOrScatter == 'Scatter':

    for i in range(len(Ys)):

      SortedY = np.sort(Ys[i],0)
      UpY = SortedY[75*Iters//100,:]
      DownY = SortedY[25*Iters//100,:]

      ax.scatter(X, np.median(SortedY, 0), label = Labels[i], color = Colors[i], s = 50, marker = Markers[i])
      ax.scatter(X, UpY, color = Colors[i], marker = '_', s = 150)
      ax.scatter(X, DownY, color = Colors[i], marker = '_', s = 150)

      for j in range(len(X)):

          x = [X[j], X[j]]
          y = [UpY[j], DownY[j]]
          ax.plot(x, y, color = Colors[i], zorder = 1)

  elif LineOrScatter == 'Line':

    for i in range(len(Ys)):

      SortedY = np.sort(Ys[i],0)
      UpY = SortedY[75*Iters//100,:]
      DownY = SortedY[25*Iters//100,:]

      ax.plot(X, np.median(SortedY, 0), label = Labels[i], color = Colors[i], marker = Markers[i])
      ax.fill_between(X, UpY, DownY, color = Colors[i], alpha = 0.25)

  if IsXTicks == 1:
    ax.set_xticks(XTicks)

  fig.legend(loc=Loc, fontsize = 14)

  if IsLogScale == 1:
    ax.set_yscale('log')
    # ax.set_xscale('log')
    # k = 1

  if SaveFig == 1:
    plt.savefig('Tightness 1Q.pdf')

  fig.show()

# Numerics

### Notation
We use the following notation \\
`Rhos` : the collection of states in the ensemble \\
`p`: The probability vector over the ensemble \\
`Sigma` (and other variants of `Sigma`) : The optimal state. \\
`Fid` (and other variants of `Fid`) : Optimal average fidelity. \\
`n`: Number of states in the ensemble $n$. \\
`d`: Dimension of the states $d$. \\
`r`: Rank of the states $r$. For fixed point algorithms to work, $r = d$ must hold. 



## Minimal working examples for Optimal average fidelity

In [ ]:
# Using Original SDP
n = 5
d = 2
r = 2

Rhos = GenStates(n, d, d)
p = ProbVec(n)

FidOriSDP, SigmaOriSDP = BayesSDPFidOri(Rhos, p, epsi = 1e-6)[0:2] # Using Original SDP

FidOriSDP

0.9661729538797927

In [ ]:
# Using alternate SDP
n = 10
d = 3
r = 3

Rhos = GenStates(n, d, d)
p = ProbVec(n)

FidAltSDP, SigmaAltSDP = BayesSDPFid(Rhos, p, epsi = 1e-6)[0:2] # Using Alternate SDP

FidAltSDP

0.926205975413983

In [ ]:
# Using Omega FP algorithm

n = 20
d = 10
r = 10

Rhos = GenStates(n, d, d)
p = ProbVec(n)

FidOmegaFP, SigmaOmegaFP = OmegaEstimator(Rhos, p, eps = 1e-6)[0:2] # Using Omega FP algorithm 

FidOmegaFP

(0.9295334307247842-2.7980391656042437e-16j)

In [ ]:
# Using Lambda FP algorithm 

n = 20
d = 5
r = 5

Rhos = GenStates(n, d, d)
p = ProbVec(n)

FidLambdaFP, SigmaLambdaFP = LambdaEstimator(Rhos, p, eps = 1e-6)[0:2] # Using Lambda FP algorithm 

FidLambdaFP

(0.929841095196262+1.842367045337236e-18j)

In [ ]:
# Commuting Estimator 

n = 20
d = 5
r = 5

Rhos = GenStates(n, d, d)
p = ProbVec(n)

SigmaComm = CommEstimator(Rhos, p) # Using Lambda FP algorithm 

AverageFidelity(SigmaComm, Rhos, p)

(0.9208440235313925-3.438407645407128e-16j)

In [ ]:
# Comparing all 4 methods over the same ensemble
n = 5
d = 2
r = 2

Rhos = GenStates(n, d, d)
p = ProbVec(n)

FidOriSDP, SigmaOriSDP = BayesSDPFidOri(Rhos, p, epsi = 1e-6)[0:2] # Using Original SDP

FidAltSDP, SigmaAltSDP = BayesSDPFid(Rhos, p, epsi = 1e-6)[0:2] # Using Alternate SDP

FidOmegaFP, SigmaOmegaFP = OmegaEstimator(Rhos, p, eps = 1e-6)[0:2] # Using Omega FP algorithm 

FidLambdaFP, SigmaLambdaFP = LambdaEstimator(Rhos, p, eps = 1e-6)[0:2] # Using Lambda FP algorithm 

print(np.allclose(FidOriSDP, FidOmegaFP), np.allclose(FidOriSDP, FidAltSDP), np.allclose(FidOriSDP, FidLambdaFP))

print(np.allclose(SigmaOriSDP, SigmaOmegaFP), np.allclose(SigmaAltSDP, SigmaOmegaFP), np.allclose(SigmaLambdaFP, SigmaOmegaFP))

# As the results show, all methods lead to the same solution

True True True
True True True
